# Dekoratorji

V Pythonu lahko definiramo funkcijo, ki vrača funkcijo.

In [ ]:
import time
from functools import wraps

def stopaj(fun):
    @wraps(fun)
    def stopaj_fun(*largs, **kwargs):
        zacetek = time.time()
        rez = fun(*largs, **kwargs)
        konec = time.time()
        args = ', '.join((*map(str, largs), *(f'{k}={v}' for k, v in kwargs.items())))
        print(f"Klic funkcije {fun.__name__}({args}) je trajal {1000 * (konec - zacetek):.2f} ms.")
        return rez
    return stopaj_fun

In [ ]:
@stopaj
def vsota_pocasna(n):
    """
    Izračunaj 1 + 2 + ... + n s prištevanjem v zanki for.
    """
    vsota = 0
    for i in range(n+1):
        vsota += i
    return vsota

In [ ]:
@stopaj
def vsota_hitra(n):
    """
    Izračunaj 1 + 2 + ... + n s formulo n*(n+1)/2.
    """
    return (n * (n+1)) // 2

In [ ]:
@stopaj
def primerjaj(m, n):
    return vsota_hitra.__wrapped__(m) == vsota_pocasna.__wrapped__(n)

In [ ]:
vsota_pocasna(10**7)

In [ ]:
vsota_hitra(10**7)

In [ ]:
primerjaj(n=10**7, m=10**6)

In [ ]:
vsota_pocasna?

# Knjižnica `bottle`

Zapišimo vzorčni spletni vmesnik s knjižnico `bottle`.

In [1]:
import bottle
import os
from IPython.display import display, Markdown

SKRIVNOST = 'nekaj, kar bo zelo težko uganiti!!!! djnskfndkjfnsd'
BOTTLE_PORT = 8080

@bottle.get('/static/<datoteka:path>')
def static(datoteka):
    return bottle.static_file(datoteka, root='static')

@bottle.get('/')
def index():
    uporabnik = bottle.request.get_cookie('uporabnik', secret=SKRIVNOST)
    return bottle.template('index.html', uporabnik=uporabnik)

@bottle.get('/pozdravi/<ime>/')
def pozdravi(ime):
    return f'Živjo, <strong>{ime}<strong>!'

@bottle.get('/kvadriraj/<n:int>')
def kvadriraj(n):
    return f'{n}^2 = {n**2}'

@bottle.get('/pozdravi_predloga/<ime>/')
def pozdravi_predloga(ime):
    return bottle.template('pozdravi.html', ime=ime)

@bottle.get('/primer')
def primer():
    return bottle.template('primer.html')

@bottle.get('/obrazec')
def obrazec():
    uporabnik = bottle.request.get_cookie('uporabnik', secret=SKRIVNOST)
    return bottle.template('obrazec.html', uporabnik=uporabnik)

@bottle.post('/obrazec/')
def obrazec_post():
    uporabnik = bottle.request.forms.uporabnik
    bottle.response.set_cookie('uporabnik', uporabnik, secret=SKRIVNOST, path='/')
    bottle.redirect('/obrazec')

@bottle.post('/odjava/')
def odjava():
    bottle.response.delete_cookie('uporabnik', path='/')
    bottle.redirect('/obrazec')

@bottle.get('/sestej/')
def sestej():
    a = bottle.request.query.a
    b = bottle.request.query.b
    return f'{a} + {b} = {a + b}'

prefix = os.environ.get('JUPYTERHUB_SERVICE_PREFIX')
if prefix:
    display(Markdown(f"[Klikni za zagon aplikacije]({prefix}/proxy/{BOTTLE_PORT}/)"))
bottle.run(debug=True, port=BOTTLE_PORT)

Bottle v0.13-dev server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [15/Dec/2023 12:57:59] "GET /obrazec/ HTTP/1.1" 405 747
127.0.0.1 - - [15/Dec/2023 12:58:01] "GET / HTTP/1.1" 200 343
127.0.0.1 - - [15/Dec/2023 12:58:02] "GET /obrazec HTTP/1.1" 200 575
127.0.0.1 - - [15/Dec/2023 12:58:05] "GET /sestej/?a=2&b=3 HTTP/1.1" 200 10
127.0.0.1 - - [15/Dec/2023 12:58:10] "POST /obrazec/ HTTP/1.1" 303 0
127.0.0.1 - - [15/Dec/2023 12:58:10] "GET /obrazec/ HTTP/1.1" 405 747
127.0.0.1 - - [15/Dec/2023 12:58:14] "POST /obrazec/ HTTP/1.1" 303 0
127.0.0.1 - - [15/Dec/2023 12:58:14] "GET /obrazec/ HTTP/1.1" 405 747
